In [1]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
#from KaggleWord2VecUtility import KaggleWord2VecUtility
#from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np

import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk.data
import logging
from gensim.models import Word2Vec

In [26]:
class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences

In [3]:
ls

300features_40minwords_100context  labeledTrainData.tsv.zip           testData.tsv.zip
NLP Word2Vec.ipynb                 sampleSubmission (1).csv           unlabeledTrainData.tsv
labeledTrainData.tsv               testData.tsv                       unlabeledTrainData.tsv.zip


In [28]:
if __name__ == '__main__':
    train = pd.read_csv('labeledTrainData.tsv', \
                        header=0, delimiter="\t", quoting=3)
    test = pd.read_csv('testData.tsv', \
                       header = 0, delimiter="\t", quoting = 3)
    
    #nltk.download()
    print "Create Review List"
    clean_train_reviews = []
    for i in xrange(0, len(train["review"])):
        clean_train_reviews.append("".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))
    
    print "Initilizing Vectorizer"
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, \
                                 preprocessor = None, stop_words = None, \
                                 max_features = 5000)
    
    print "Fitting Vectorizer"
    train_data_features = vectorizer.fit_transform(clean_train_reviews)
    train_data_features = train_data_features.toarray()
    
    print "Fitting Random Forest"
    forest = RandomForestClassifier(n_estimators = 100)
    forest = forest.fit(train_data_features, train["sentiment"])
    
    clean_test_reviews = []
    for i in xrange(len(test["review"])):
        clean_test_reviews.append("".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True)))
    
    test_data_features = vectorizer.transform(clean_test_reviews)
    test_data_features = test_data_features.toarray()
    
    print "Making Predictions"
    predictions = forest.predict(test_data_features)
    
    output = pd.DataFrame(data={"id":test["id"], "sentiment":predictions})

                            
                    
    print output[:5]

Create Review List


KeyboardInterrupt: 

In [5]:
print sum(output['sentiment'])

24981


In [6]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec,model[word])
            
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [7]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    for review in reviews:
        if counter%1000. == 0.:
            print "Review %d of %d" %(counter, len(reviews))
        
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter += 1
    return reviewFeatureVecs

In [27]:
def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append(KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True))
    return clean_reviews

In [9]:
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter="\t",\
                    quoting = 3)
test = pd.read_csv('testData.tsv', header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv('unlabeledTrainData.tsv', header=0,\
                    delimiter="\t", quoting=3)

print "Read %d labeled train reviews, %d labeled tst reviews, "\
    "and %d unlabeled reviews\n" %(train["review"].size, \
    test["review"].size, unlabeled_train["review"].size)

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Read 25000 labeled train reviews, 25000 labeled tst reviews, and 50000 unlabeled reviews



In [10]:
downsampling = 1e-3

In [11]:
# Set up list of sentences
sentences = []
for review in train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
for review in unlabeled_train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
                    level=logging.INFO)

num_features = 300
min_word_count = 40
num_workers = 4
context=10
downsampling = 1e-3
         

/Users/jeremynixon/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/jeremynixon/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/jeremynixon/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful 

In [12]:
print "Training Word2Vec Model"
model = Word2Vec(sentences, workers=num_workers, \
        size=num_features, min_count = min_word_count, \
        window = context, sample = downsampling, seed=1)

model.init_sims(replace=True)
model_name = "300features_40minwords_100context"
model.save(model_name)

model.doesnt_match("man woman child kitchen".split())
model.doesnt_match("france england germany berlin".split())
model.doesnt_match("paris berlin london austria".split())
model.most_similar("man")
model.most_similar("queen")
model.most_similar("awful")

Training Word2Vec Model


[(u'terrible', 0.6566125154495239),
 (u'dreadful', 0.6504294276237488),
 (u'horrible', 0.6498185396194458),
 (u'atrocious', 0.5685333013534546),
 (u'horrendous', 0.530869722366333),
 (u'laughable', 0.5282012224197388),
 (u'abysmal', 0.520856499671936),
 (u'horrid', 0.5092235803604126),
 (u'amateurish', 0.49969422817230225),
 (u'embarrassing', 0.49759671092033386)]

In [36]:
model.most_similar('horrid')

[(u'horrendous', 0.6692837476730347),
 (u'atrocious', 0.6487591862678528),
 (u'abysmal', 0.6437088251113892),
 (u'dreadful', 0.6213812828063965),
 (u'appalling', 0.5963515639305115),
 (u'amateurish', 0.5912222266197205),
 (u'lousy', 0.5728222131729126),
 (u'uninspired', 0.5604196786880493),
 (u'laughably', 0.5595484972000122),
 (u'horrible', 0.5568745136260986)]

In [13]:
         
# Create average feature vectors
trainDataVecs = getAvgFeatureVecs(getCleanReviews(train), model, num_features)
testDataVecs = getAvgFeatureVecs(getCleanReviews(test), model, num_features)
         
forest = RandomForestClassifier(n_estimators = 100)
print "Fit Forest"
forest = forest.fit(trainDataVecs, train["sentiment"])

print "Evaluate Model"
result = forest.predict(testDataVecs)
result_df = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
print result_df[:5]
         

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 o

In [14]:
model.doesnt_match("france england germany".split())

'germany'

In [15]:
model.most_similar("gorgeous")

[(u'beautiful', 0.6591428518295288),
 (u'stunning', 0.6472392082214355),
 (u'lovely', 0.5601093173027039),
 (u'fabulous', 0.5319967865943909),
 (u'stunningly', 0.5270993709564209),
 (u'magnificent', 0.5060358047485352),
 (u'exquisite', 0.5053349733352661),
 (u'ravishing', 0.49005264043807983),
 (u'scenery', 0.4878876805305481),
 (u'breathtaking', 0.48135805130004883)]

In [16]:
model.most_similar("erotic")

[(u'explicit', 0.5603921413421631),
 (u'homo', 0.5117158889770508),
 (u'erotica', 0.5017858743667603),
 (u'steamy', 0.4718930125236511),
 (u'provocative', 0.46233004331588745),
 (u'sex', 0.4541817903518677),
 (u'sensual', 0.45085692405700684),
 (u'softcore', 0.4478890597820282),
 (u'graphic', 0.44152939319610596),
 (u'lesbian', 0.43881791830062866)]

In [17]:
print type(model.syn0)
print model.syn0.shape
print model.syn0[:5]

<type 'numpy.ndarray'>
(16490, 300)
[[ 0.05509147  0.08054097 -0.09385615 ...,  0.05974855 -0.01102365
  -0.13534875]
 [-0.00925186 -0.06010386 -0.00087138 ...,  0.02786387  0.00553402
  -0.10258783]
 [-0.08020471  0.02614686  0.00953095 ..., -0.02849959 -0.00536558
  -0.11704617]
 [ 0.06865267  0.05989339 -0.02981789 ..., -0.06382021 -0.00667759
   0.07546818]
 [ 0.0447268  -0.12028047  0.02066791 ...,  0.03945505 -0.02332433
   0.11325554]]


In [29]:
model["beautiful"]

array([-0.08302618,  0.0845832 , -0.02876051,  0.00547742,  0.05185456,
        0.05171555,  0.01204805, -0.05949849,  0.00616412, -0.11074502,
        0.08529584, -0.01797769, -0.0185579 , -0.0853605 ,  0.00720717,
       -0.00132107, -0.06745391,  0.03170697, -0.07726242, -0.06286061,
        0.05130217, -0.00735706,  0.08598652,  0.0924549 , -0.03147062,
       -0.00824225,  0.02680037, -0.04916568,  0.01619515, -0.01466607,
       -0.03067497,  0.03071502, -0.05392663, -0.08849475, -0.06392509,
       -0.02367905, -0.11715134, -0.02845477, -0.04064883, -0.01474903,
        0.01794586, -0.10553849,  0.01120909,  0.08259062, -0.02174144,
       -0.04461825,  0.01456434,  0.03949435, -0.02223836, -0.0489157 ,
        0.07372109,  0.10857644,  0.12177243, -0.04384258, -0.01848895,
       -0.01638948, -0.00144819, -0.05726345, -0.00023927,  0.02017483,
        0.02872772, -0.08188444,  0.08449091, -0.0180668 , -0.01195569,
        0.03048421, -0.07581793,  0.01857644,  0.04885089, -0.06

In [19]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
testDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)


NameError: name 'review_to_wordlist' is not defined

In [20]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [32]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
def functionx( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)
    
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( functionx( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( functionx( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [33]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [34]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Time taken for K Means clustering:  394.29006505 seconds.

Cluster 0
[u'politically', u'correct', u'incorrect']

Cluster 1
[u'sickening', u'repellent', u'cuteness', u'disturbingly', u'insanely', u'relentlessly', u'impossibly', u'backbone']

Cluster 2
[u'stuffing', u'warnings', u'rants', u'coin', u'rot']

Cluster 3
[u'nickname', u'priestess', u'frankie', u'rents', u'inherited', u'owns', u'ailing', u'inherits', u'carlitos', u'banker', u'buys', u'pupils', u'employee']

Cluster 4
[u'ally', u'lizzie', u'duchess', u'gwen', u'sheedy']

Cluster 5
[u'merit']

Cluster 6
[u'sidekicks', u'cronies', u'tosses', u'crafty', u'dastardly', u'cohorts', u'silk', u'opponent', u'troy', u'associates']

Cluster 7
[u'fairytale', u'retelling', u'stardust', u'underdog', u'fable', u'whimsical', u'yarn', u'actuality', u'timeless']

Cluster 8
[u'counted', u'prequels', u'bsg', u'originals', u'braveheart', u'newer']

Cluster 9
[u'ussr']


In [35]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
